support vector machine

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss, classification_report, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

kyph = pd.read_csv('../Datasets/cases/Kyphosis/Kyphosis.csv')
le = LabelEncoder()
kyph.head()

,Kyphosis,Age,Number,Start
0,absent,71,3,5
1,absent,158,3,14
2,present,128,4,5
3,absent,2,5,1
4,absent,1,4,15


In [3]:
kyph['Kyphosis'] = le.fit_transform(kyph['Kyphosis'])
y = kyph['Kyphosis']
X = kyph.drop('Kyphosis', axis=1)

In [4]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=25, stratify=y)

In [15]:
mm = MinMaxScaler().set_output(transform='pandas')
X_trn_mm = mm.fit_transform(X_train)
X_tst_mm = mm.transform(X_test)

std = StandardScaler().set_output(transform='pandas')
X_trn_std = std.fit_transform(X_train)
X_tst_std = std.transform(X_test)

svm import: support vector classifier

In [16]:
from sklearn.svm import SVC

svm on unscaled

In [17]:
svm = SVC(kernel='linear', C=1)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(classification_report(y_test,y_pred))
print()
print(f1_score(y_test, y_pred, pos_label=1))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93        20
           1       1.00      0.40      0.57         5

    accuracy                           0.88        25
   macro avg       0.93      0.70      0.75        25
weighted avg       0.90      0.88      0.86        25


0.5714285714285714


using minmax scaler, zero_division to suppress warning

In [18]:
svm = SVC(kernel='linear', C=1)
svm.fit(X_trn_mm, y_train)
y_pred = svm.predict(X_tst_mm)
print(classification_report(y_test,y_pred, zero_division=0.0))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89        20
           1       0.00      0.00      0.00         5

    accuracy                           0.80        25
   macro avg       0.40      0.50      0.44        25
weighted avg       0.64      0.80      0.71        25



standard scaler svm

In [19]:
svm = SVC(kernel='linear', C=1)
svm.fit(X_trn_std, y_train)
y_pred = svm.predict(X_tst_std)
print(classification_report(y_test,y_pred, zero_division=0.0))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93        20
           1       1.00      0.40      0.57         5

    accuracy                           0.88        25
   macro avg       0.93      0.70      0.75        25
weighted avg       0.90      0.88      0.86        25



why so much difference between two scalers?

different C parameter values

In [10]:
Cs = np.linspace(1e-3, 5, 20)

scores = []
for C in Cs:
    svm = SVC(kernel='linear', C=C)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    
    scores.append([C, f1_score(y_test, y_pred, pos_label=1)])

scores = pd.DataFrame(scores, columns=['C', 'F1_score for present (1)'])
scores.sort_values(by=scores.columns[1], ascending=False)

,C,F1_score for present (1)
1,0.264105,0.571429
2,0.527211,0.571429
3,0.790316,0.571429
4,1.053421,0.571429
12,3.158263,0.571429
5,1.316526,0.571429
6,1.579632,0.571429
7,1.842737,0.571429
8,2.105842,0.571429
9,2.368947,0.571429


looop on minmax scaled data

In [11]:
Cs = np.linspace(1e-3, 5, 20)

scores = []
for C in Cs:
    svm = SVC(kernel='linear', C=C)
    svm.fit(X_trn_mm, y_train)
    y_pred = svm.predict(X_tst_mm)
    
    scores.append([C, f1_score(y_test, y_pred, pos_label=1)])

scores = pd.DataFrame(scores, columns=['C', 'F1_score for present (1)'])
scores.sort_values(by=scores.columns[1], ascending=False)

,C,F1_score for present (1)
19,5.000000,0.571429
18,4.736895,0.333333
0,0.001000,0.000000
1,0.264105,0.000000
3,0.790316,0.000000
2,0.527211,0.000000
4,1.053421,0.000000
5,1.316526,0.000000
8,2.105842,0.000000
9,2.368947,0.000000


looooop on standard scaled data

In [13]:
Cs = np.linspace(1e-3, 5, 20)

scores = []
for C in Cs:
    svm = SVC(kernel='linear', C=C)
    svm.fit(X_trn_std, y_train)
    y_pred = svm.predict(X_tst_std)
    
    scores.append([C, f1_score(y_test, y_pred, pos_label=1)])

scores = pd.DataFrame(scores, columns=['C', 'F1_score for present (1)'])
scores.sort_values(by=scores.columns[1], ascending=False)

,C,F1_score for present (1)
3,0.790316,0.5
2,0.527211,0.5
5,1.316526,0.5
4,1.053421,0.5
12,3.158263,0.5
13,3.421368,0.5
6,1.579632,0.5
7,1.842737,0.5
8,2.105842,0.5
9,2.368947,0.5


sir using column transformer and pipeline

In [14]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline


use scaling + model in pipeline

In [20]:
std = StandardScaler()
svm = SVC(kernel='linear', C=C)
Cs = np.linspace(1e-3, 5, 20)

scores = []
for C in Cs:
    pipe = Pipeline(
        [('StandardScaler', std),
         ('SupportVectorMachine', svm)]
    )
    pipe.fit(X_trn_std, y_train)
    y_pred = pipe.predict(X_tst_std)
    
    scores.append([C, f1_score(y_test, y_pred, pos_label=1)])

scores = pd.DataFrame(scores, columns=['C', 'F1_score for present (1)'])
scores.sort_values(by=scores.columns[1], ascending=False)

,C,F1_score for present (1)
0,0.001000,0.571429
1,0.264105,0.571429
2,0.527211,0.571429
3,0.790316,0.571429
4,1.053421,0.571429
5,1.316526,0.571429
6,1.579632,0.571429
7,1.842737,0.571429
8,2.105842,0.571429
9,2.368947,0.571429
